In [ ]:
%%writefile Sequential.cu
#include <stdio.h>
#include <malloc.h>
#include <math.h>
#define M 10
#define Nt 10
#define dxy 0.1
#define dt 0.01
//==============================
float funct1(int i, int j) {
  return sin(M_PI*(i*dxy + j*dxy));
}
float funct2(int i, int j) {
  return cos(M_PI*(i*dxy + j*dxy));
}
//==============================
void InputData(float *U, float *V, float *C) {
  int i,j,t;
  for (j=0; j<M; j++) {
    for (i=0; i<M; i++) {
      *(U + j*M + i) = funct1(i,j);
      *(V + j*M + i) = dt*funct2(i,j) + *(U + j*M + i);
    }
  }
  for (t=0; t<Nt; t++) {
    *(C+t) = 1/2;
  }
}
//===============================
void Computing(float *A, float *B, float *C, int t) {
  int l;
  float left, right, bottom, top, center1, center2;
  for (l=0; l<M*M; l++) {
      left  =  (l%M == 0)       ? 0 : *(B + l - 1);
      right =  (l%M == M-1)    ? 0 : *(B + l + 1);
      bottom = (l >= M*(M-1))  ? 0 : *(B + l - M);
      top   =  (l <= M-1)       ? 0 : *(B + l + M);
      center1 = *(B + l);
      center2 = *(A + l);
      *(A + l) = 2*center1 - center2 + ((dt * dt * (*(C + t)) * (*(C + t))) / (dxy * dxy)) * (left + right + bottom + top - 4*center1);
  }
}
//===============================
int main() {
  float *U, *V, *C;
  int t;
  U = (float *) malloc (M*M*sizeof(float));
  V = (float *) malloc (M*M*sizeof(float));
  C = (float *) malloc (Nt*sizeof(float));
  InputData(U, V, C);
  for (t=2; t<Nt; t++) {
    if (t%2==0) Computing(U, V, C, t);
    else Computing(V, U, C, t);
  }

  //Show results 
  if (Nt % 2 == 1) {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        printf("%f ", *(U + j * M + i));
      }
      printf("\n");
    }
  } else {
    for (int j = 0; j < M; j++) {
      for (int i = 0; i < M; i++) {
        printf("%f ", *(V + j * M + i));
      }
      printf("\n");
    }
  }
  free(U); free(V);
}

In [ ]:
!nvcc Sequential.cu -lm

In [ ]:
!./a.out